# Saleforce data objects and logs

Digging through the Salesforce data.

> Questions that you will be able to answer at the end of this notebook.
>1. [x] What are Salesforce objects?
>2. [x] Where are the Salesforce objects?
>3. [x] What are Lightning event logs?
>4. [x] Where are the Lightning logs?
>5. [x] What are the common features on lightning logs?
>6. [x] How can we join different lightning log tables?
>7. [x] How can we get report object Name and ID from lightning logs?

In [1]:
import os
import pandas as pd

Listing our sample data.

In [2]:
os.listdir('data/Salesforce')

['ELF', 'Objects']

We have two types of data samples.

- `Objects`- Salesforce objects.
- `ELF` - Event log files.


`Objects` are the entities created within the salesforce platform. It contains information about existing reports, dashboards, users, event log files, field definitions etc. The `ELF` are the logs created from these reports/dashboards, whether to capture user interactions, or to capture errors or analytical information about performance and pageview.


## Lightning event logs

There is a wide variety of event-related logs. But we will only be considering the following types of events in our analysis.

In [3]:
os.listdir('data/Salesforce/ELF')

['LightningError',
 'LightningInteraction',
 'LightningPageView',
 'LightningPerformance',
 'Report']

All related to logs coming from the `Lightning platform`.

- `Report` - User run reports.
- `LightningPageView` - Page load time.
- `LightingPerformance` - Runtime performance.
- `LightingError`
- `LightingInteraction`

## Salesforce reports

To get the files where these logs are saved and what they refer to we need the following samples.

In [4]:
os.listdir('data/Salesforce/Objects')

['DataLoad_Exception_Log__c_5k_sample.csv',
 'EventDefinition_1k_samples.csv',
 'EventLogFile_all_sample.csv',
 'ExceptionLogger__c_5k_sample.csv',
 'FieldDefinition_all_sample.csv',
 'ReportEvent_50k_Sample.csv',
 'Report_all_sample.csv']

Below the most important ones.

- `Report` - Report objects.
- `EventLogFile` - Defines the logs files based on event types.
- `FieldDefinition` - Defines the fields and data typesfor the objects.

## 1. Exploratory Data Analysis
---

### 1.1. Salesforce objects

Loading report objects.

In [5]:
report_objects = pd.read_csv("./data/Salesforce/Objects/Report_all_sample.csv")

In [6]:
report_objects.shape # (number of reports, number of report features)

(17397, 18)

In [7]:
list(report_objects.columns)

['CreatedById',
 'CreatedDate',
 'Description',
 'DeveloperName',
 'FolderName',
 'Format',
 'Id',
 'IsDeleted',
 'LastModifiedById',
 'LastModifiedDate',
 'LastReferencedDate',
 'LastRunDate',
 'LastViewedDate',
 'Name',
 'NamespacePrefix',
 'OwnerId',
 'ReportTypeApiName',
 'SystemModstamp']

From there we have access to various information, including the `Name` and `Id` of the report. We also have the `LastRunDate` of its execution, important information for our future analyses.

In [8]:
interest_columns = ['Id', 'Name', 'FolderName', 'Format', 'LastRunDate']
report_objects[interest_columns].sample(5)

,Id,Name,FolderName,Format,LastRunDate
15119,00O2R000003s29hUAA,Billing Milestones for Custom (AS EDIT),Finance AMER SME,Tabular,2021-06-14T14:56:44.000Z
7810,00O6P000000NvsaUAC,Copy of S&AS Planned Queues,BTT - Silvano Private Reports (shared),Tabular,2022-02-23T10:23:38.000Z
9884,00O2R000003s2LYUAY,My Team Forecast Rev by Month,PDM Operations Dashboard (Public),Summary,2022-06-19T14:00:02.000Z
7596,00O6P0000012iwaUAA,AMER - Dish Open Cases By ESG Products,ISG_NA_BusinessExcellence,Summary,2022-05-12T19:09:20.000Z
3372,00O1P000003Qi4zUAC,Q3 Timecard Submissions,Utilization Reporting,Matrix,2022-02-02T01:23:12.000Z


### 1.2. Lightning event logs

Loading Lightning event logs.

In [9]:
ltng_elf_path = 'data/Salesforce/ELF'
ltng_logs = {}

for event_log_file in os.listdir(ltng_elf_path):
    log_path = os.path.join(ltng_elf_path, event_log_file)
    log_samples = os.listdir(log_path)
    random_logs = sorted(log_samples)[0]
    print(random_logs)
    log_name = random_logs.split("_", 1)[-1][:-4]
    if log_name.startswith('Lightning'):
        log_name = log_name[len('Lightning'):]
    ltng_logs[log_name] = pd.read_csv(log_path +"/"+ random_logs, nrows=1000, low_memory=False)

2022-06-01_LightningError.csv
2022-06-04_LightningInteraction.csv
2022-06-04_LightningPageView.csv
2022-06-04_LightningPerformance.csv
2022-06-04_Report.csv


In [10]:
ltng_logs.keys()

dict_keys(['Error', 'Interaction', 'PageView', 'Performance', 'Report'])

All event log features that we are considering follows the most recent Salesforce developer documentation at `version 55`.

#### 1.2.1. Report event

In [11]:
report_event_cols = ltng_logs['Report'].columns
# sorted(report_event_cols)

In [12]:
report_event_cols.shape # number of feature columns

(31,)

#### 1.2.2. Performance

In [13]:
performance_event_cols = ltng_logs['Performance'].columns
sorted(performance_event_cols)

['APP_NAME',
 'BROWSER_NAME',
 'BROWSER_VERSION',
 'CLIENT_GEO',
 'CLIENT_ID',
 'CLIENT_IP',
 'CONNECTION_TYPE',
 'DEVICE_ID',
 'DEVICE_MODEL',
 'DEVICE_PLATFORM',
 'DEVICE_SESSION_ID',
 'DURATION',
 'EVENT_TYPE',
 'LOGIN_KEY',
 'ORGANIZATION_ID',
 'OS_NAME',
 'OS_VERSION',
 'PAGE_START_TIME',
 'REQUEST_ID',
 'SDK_APP_TYPE',
 'SDK_APP_VERSION',
 'SDK_VERSION',
 'SESSION_KEY',
 'TIMESTAMP',
 'TIMESTAMP_DERIVED',
 'UI_EVENT_ID',
 'UI_EVENT_SOURCE',
 'UI_EVENT_TIMESTAMP',
 'UI_EVENT_TYPE',
 'USER_AGENT',
 'USER_ID',
 'USER_ID_DERIVED',
 'USER_TYPE']

In [14]:
performance_event_cols.shape

(33,)

#### 1.2.3. PageView

In [15]:
pageview_event_cols = ltng_logs['PageView'].columns
# sorted(pageview_event_cols)

In [16]:
pageview_event_cols.shape

(50,)

#### 1.2.4. Error

In [17]:
error_event_cols = ltng_logs['Error'].columns
# sorted(error_event_cols)

In [18]:
error_event_cols.shape

(39,)

#### 1.2.5. Interaction

In [19]:
interaction_event_cols = ltng_logs['Interaction'].columns
# sorted(inter_event_cols)

In [20]:
interaction_event_cols.shape

(46,)

### 1.3. Common features

In [21]:
common_columns = set(
    list(report_event_cols) + 
    list(performance_event_cols) +
    list(pageview_event_cols) +
    list(error_event_cols) + 
    list(interaction_event_cols))

In [22]:
from collections import defaultdict

ltng_log_features = defaultdict(list)

for feature in common_columns:
    ltng_log_features['Feature'].append(feature)
    for event_type in ltng_logs.keys():
        ltng_log_features[event_type].append(feature in list(ltng_logs[event_type].columns))

Here we have the list of all the features found in the logs. We can check which of them are present in each report event type.

In [23]:
pd.DataFrame(ltng_log_features)

,Feature,Error,Interaction,PageView,Performance,Report
0,CLIENT_GEO,True,True,True,True,False
1,PAGE_ENTITY_ID,True,True,True,False,False
2,DEVICE_SESSION_ID,True,True,True,True,False
3,SDK_VERSION,True,True,True,True,False
4,OS_VERSION,True,True,True,True,False
...,...,...,...,...,...,...
70,SESSION_KEY,True,True,True,True,True
71,EVENT_TYPE,True,True,True,True,True
72,UI_EVENT_TIMESTAMP,True,True,True,True,False
73,NUMBER_COLUMNS,False,False,False,False,True


In [24]:
common_log_features = pd.DataFrame(ltng_log_features)

In [25]:
# common_log_features.to_csv('datasets/common_features_in_the_logs.csv', index=False)

#### 1.3.1. Finding the relationship between these logs

Let's see how can we associate these logs by looking at the `REPORT_ID` column.

In [26]:
common_log_features[common_log_features.Feature.str.startswith('REPORT')]

,Feature,Error,Interaction,PageView,Performance,Report
15,REPORT_ID_DERIVED,False,False,False,False,True
43,REPORT_ID,False,False,False,False,True


It looks like we don't have the **report identifier** in the Error, Interaction, PageView and Performance logs. But if we look at the endpoints present in the `PAGE_URL` column in all logs except for Performance. We can see that this informantion can be rescued.

In [27]:
ltng_logs['Error'].PAGE_URL.sample()

646    /lightning/o/DispatchPendingApproval__c/list?f...
Name: PAGE_URL, dtype: object

In [28]:
ltng_logs['Interaction'].PAGE_URL.sample()

619    /lightning/r/Case/5006P000005RnuoQAC/view
Name: PAGE_URL, dtype: object

In [29]:
ltng_logs['PageView'].PAGE_URL.sample()

458    /lightning/r/Task/00T6P00000HpXGQUA3/view?ws=%...
Name: PAGE_URL, dtype: object

We can also check our common features table.

In [30]:
common_log_features[common_log_features.Feature=='PAGE_URL']

,Feature,Error,Interaction,PageView,Performance,Report
11,PAGE_URL,True,True,True,False,False


>**Obs. 1:** Its important to realize that we are interested in run reports. In this way, only the endpoints related to the execution will be condidered in our analysis. To identify these endpoints we look for those with the prefix `/ligtning/r/` indicating a run report resource.

##### .. but what about the Performance logs?

We can notice that there is a subset of common features among the logs that do not relate the report identifier, such as:

- `USER_ID`
- `USER_ID_DERIVED`
- `SESSION_KEY`
- `LOGIN_KEY`
- `ORGANIZATION_ID`
- `CLIENT_IP`

In [31]:
common_subset_features = ['USER_ID', 'USER_ID_DERIVED', 'SESSION_KEY', 'LOGIN_KEY', 'ORGANIZATION_ID', 'CLIENT_IP']

In [32]:
common_log_features.set_index('Feature').loc[common_subset_features]

,Error,Interaction,PageView,Performance,Report
Feature,,,,,
USER_ID,True,True,True,True,True
USER_ID_DERIVED,True,True,True,True,True
SESSION_KEY,True,True,True,True,True
LOGIN_KEY,True,True,True,True,True
ORGANIZATION_ID,True,True,True,True,True
CLIENT_IP,True,True,True,True,True


If we consider all these features, we can take for granted the possibility of retrieving the identifier of a report associated with the performance log. Note that we can consider the `USER_ID` or `USER_ID_DERIVED`, let keep the second.

In [33]:
common_subset_features.remove('USER_ID')

Retrieving the `REPORT_ID_DERIVED` through the performance logs.

In [34]:
pd.merge(
    left=ltng_logs['Performance'], 
    right=ltng_logs['Report'], 
    on=common_subset_features).REPORT_ID_DERIVED.value_counts()

00O2R000004Anm0UAC    96
00O2R0000043lHLUAY    30
00O2R0000043lHKUAY    20
00O0b000006iNwdEAE    14
00O0b000004kTazEAE    14
00O6P000000ZMcoUAG     8
00O2R000003zOIAUA2     8
00O0b000004AmRrEAK     6
00O2R000004AmgKUAS     4
00O0b000005yLBGEA2     4
Name: REPORT_ID_DERIVED, dtype: int64

We can do the same to relate any pair of logs. For example, `performance-pageview`, `pageview-errors` etc.

>**Obs. 2:** Have you ever wondered why we are not considering the `TIMESTAMP` column to cross this data?
> Even in the case of logs referring to the same user, session, login key, organization and client IP. The timestamp may take more or less time to save, so they can difer.

Try to include the column `TIMESTAMP`.

In [35]:
pd.merge(
    left=ltng_logs['Performance'], 
    right=ltng_logs['Report'], 
    on=common_subset_features + ['TIMESTAMP_DERIVED']).REPORT_ID_DERIVED.value_counts()

Series([], Name: REPORT_ID_DERIVED, dtype: int64)

We can see that no intercept was achieved during the join. But note that there may be such aoccurrences with real data.

#### 1.3.2. Getting the report name

Now that we can cross information from the logs we can also get information from a certain report.

To do this, just associate any of the Lightning logs with the report object presented in the first section [Salesforce objects](#1.1.-Salesforce-objects).

For instance, let's take the `PageView logs` and show the names of the reports that took the longest since page start time. From pageview logs we want information like `DURATION`, `PAGE_START_TIME` and `PAGE_URL` (where we retrieve the report ID from). And from the report object we just want the name.

`DURATION` and `PAGE_START_TIME` are both in milliseconds.

In [36]:
pageview_cols = ['DURATION', 'PAGE_START_TIME', 'PAGE_URL']
report_obj_cols = ['Id', 'Name'] # report id and name

In [37]:
ltng_pageview = ltng_logs['PageView'][pageview_cols]
report_names = report_objects[report_obj_cols]

Removnig missing data.

In [38]:
ltng_pageview_cleaned = ltng_pageview.dropna(subset=pageview_cols)

Filtering run reports endpoints. For this we load a regular expression module.

In [39]:
import re

pattern = re.compile(r'\/lightning\/r\/(?P<report_type>[a-zA-Z]{4,})\/(?P<report_id>[0-9a-zA-Z]{18})')

def filter_run_report_endpoints(pattern, url):
    m = re.match(pattern, url)
    if m:
        return m.group('report_id')

In [40]:
ltng_pageview_cleaned['REPORT_ID_DERIVED'] =\
    ltng_pageview_cleaned.PAGE_URL.apply(lambda url: filter_run_report_endpoints(pattern, url))

C:\Users\jefer\AppData\Local\Temp\ipykernel_8932\2053880511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ltng_pageview_cleaned['REPORT_ID_DERIVED'] =\


In [41]:
ltng_pageview_filtered = ltng_pageview_cleaned.dropna(subset=['REPORT_ID_DERIVED'])

In [42]:
# ltng_pageview_filtered.head()

Making a join and sorting by `DURATION`.

In [43]:
pd.merge(left=report_names, right=ltng_pageview_filtered, left_on='Id', right_on='REPORT_ID_DERIVED')\
    .sort_values('DURATION', ascending=False)[['Name', 'DURATION']]\
    .groupby(['Name'])\
    .agg('mean')\
    .reset_index()\
    .rename(columns={'Name': 'ReportName', 'DURATION': 'AverageDuration'})\
    .head()

,ReportName,AverageDuration
0,Activities Closing Today,18885.0
1,Case Aging Report,15607.5
2,Case Load OSP,17477.0
3,"Cases with Tasks, Calendar & Reminders",19540.2
4,Chat Average Handle Time,7219.5
